In [1]:
from pyspark.sql import SparkSession, functions as F, types as T
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [4]:
print(spark.version)

3.4.1


In [5]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [19]:
film_cat_join = film_category_df.join(film_df, on="film_id", how="inner")
film_cat_name = category_df.join(film_cat_join, on="category_id", how="inner")
film_cat = film_cat_name.groupBy("name").count()
film_cat.orderBy("count",ascending=False).show()

+-----------+-----+
|       name|count|
+-----------+-----+
|     Sports|   74|
|    Foreign|   73|
|     Family|   69|
|Documentary|   68|
|  Animation|   66|
|     Action|   64|
|        New|   63|
|      Drama|   62|
|      Games|   61|
|     Sci-Fi|   61|
|   Children|   60|
|     Comedy|   58|
|     Travel|   57|
|   Classics|   57|
|     Horror|   56|
|      Music|   51|
+-----------+-----+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [89]:
film_acvt = film_df.join(film_actor_df, on="film_id", how="left").join(actor_df, on="actor_id", how="inner")
film_acvt = film_acvt.join(inventory_df, on="film_id", how="left")
film_acvt.groupBy("actor_id").count().join(actor_df, on="actor_id", how="inner").orderBy("count",ascending=False).show(10)

+--------+-----+----------+-----------+-------------------+
|actor_id|count|first_name|  last_name|        last_update|
+--------+-----+----------+-----------+-------------------+
|     107|  214|      GINA|  DEGENERES|2022-02-15 11:34:33|
|     181|  200|   MATTHEW|     CARREY|2022-02-15 11:34:33|
|     198|  192|      MARY|     KEITEL|2022-02-15 11:34:33|
|     102|  188|    WALTER|       TORN|2022-02-15 11:34:33|
|     144|  184|    ANGELA|WITHERSPOON|2022-02-15 11:34:33|
|     150|  178|     JAYNE|      NOLTE|2022-02-15 11:34:33|
|      37|  177|       VAL|     BOLGER|2022-02-15 11:34:33|
|      23|  175|    SANDRA|     KILMER|2022-02-15 11:34:33|
|      60|  170|     HENRY|      BERRY|2022-02-15 11:34:33|
|     108|  169|    WARREN|      NOLTE|2022-02-15 11:34:33|
+--------+-----+----------+-----------+-------------------+
only showing top 10 rows



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [88]:
film_rent = rental_df.join(payment_df, on="rental_id", how="inner").join(inventory_df, on="inventory_id", how="left")
film_rent = film_rent.join(film_category_df, on="film_id", how="left").join(category_df,on="category_id",how="left")
category_rent = film_rent.select("category_id","amount")
film_rent = film_rent.groupBy("category_id").sum("amount").join(category_df,on="category_id",how="inner")
film_rent = film_rent.sort("sum(amount)",ascending=False).select("name","sum(amount)").show(1)

+------+-----------------+
|  name|      sum(amount)|
+------+-----------------+
|Sports|5314.209999999848|
+------+-----------------+
only showing top 1 row



4.
Вивести назви фільмів, яких не має в inventory.

In [70]:
films_inventory = film_category_df.join(inventory_df, on="film_id", how="left")
films_inventory.groupBy('inventory_id','film_id').count()
films_not_in_inventory = films_inventory.where(films_inventory.inventory_id.isNull())
films_not_in_inventory.join(film_df,on="film_id",how="inner").select("title","description").show()

+--------------------+--------------------+
|               title|         description|
+--------------------+--------------------+
|      ALICE FANTASIA|A Emotional Drama...|
|         APOLLO TEEN|A Action-Packed R...|
|      ARGONAUTS TOWN|A Emotional Epist...|
|       ARK RIDGEMONT|A Beautiful Yarn ...|
|ARSENIC INDEPENDENCE|A Fanciful Docume...|
|   BOONDOCK BALLROOM|A Fateful Panoram...|
|       BUTCH PANTHER|A Lacklusture Yar...|
|       CATCH AMISTAD|A Boring Reflecti...|
| CHINATOWN GLADIATOR|A Brilliant Panor...|
|      CHOCOLATE DUCK|A Unbelieveable S...|
|COMMANDMENTS EXPRESS|A Fanciful Saga o...|
|    CROSSING DIVORCE|A Beautiful Docum...|
|     CROWDS TELEMARK|A Intrepid Docume...|
|    CRYSTAL BREAKING|A Fast-Paced Char...|
|          DAZED PUNK|A Action-Packed S...|
|DELIVERANCE MULHO...|A Astounding Saga...|
|   FIREHOUSE VIETNAM|A Awe-Inspiring C...|
|       FLOATS GARDEN|A Action-Packed E...|
|FRANKENSTEIN STRA...|A Insightful Char...|
|  GLADIATOR WESTWARD|A Astoundi

5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [87]:
childrens_films = category_df.filter(category_df.name == "Children").join(film_category_df,on="category_id",how="inner").select("category_id","name","film_id")
childrens_actor = childrens_films.join(film_actor_df,on="film_id",how="left")
result_actor = childrens_actor.groupBy('actor_id').count().join(actor_df,on="actor_id",how="inner")
result_actor.sort("count",ascending=False).show(3)


+--------+-----+----------+---------+-------------------+
|actor_id|count|first_name|last_name|        last_update|
+--------+-----+----------+---------+-------------------+
|      17|    7|     HELEN|   VOIGHT|2022-02-15 11:34:33|
|      66|    5|      MARY|    TANDY|2022-02-15 11:34:33|
|     140|    5|    WHOOPI|     HURT|2022-02-15 11:34:33|
+--------+-----+----------+---------+-------------------+
only showing top 3 rows



Stop Spark session:

In [17]:
spark.stop()